# Set up

In [3603]:
from worde4mde import load_embeddings
sgram_mde = load_embeddings('sgram-mde')

In [3604]:
word = 'ecore'
sgram_mde[word]

array([ 0.14674647,  0.42704162,  0.17717203,  0.05179158,  0.38020504,
       -0.00091264,  0.02094162,  0.00924711, -0.13806985,  0.1403339 ,
       -0.0953936 , -0.03681302,  0.38122246, -0.03746711, -0.17595871,
       -0.02031669,  0.23675247,  0.27676958, -0.07487358, -0.1987225 ,
        0.02015282,  0.05147164, -0.223504  ,  0.00699752, -0.06145973,
        0.00920826,  0.01364714,  0.10218145, -0.25330105, -0.14080754,
        0.2541444 ,  0.4577827 , -0.16825216, -0.08050938,  0.11404785,
       -0.38822263, -0.06395938, -0.4724689 , -0.00111705,  0.27869806,
        0.28032073,  0.02976846, -0.01675524,  0.09774698,  0.18298927,
       -0.24398401, -0.3238438 ,  0.01912955,  0.11861664,  0.08640955,
        0.18799016, -0.06891091, -0.03893414, -0.04862301,  0.02151174,
       -0.39205688, -0.23195332, -0.09532862, -0.01324714, -0.24555108,
        0.09372689, -0.21971853, -0.26171875, -0.3605202 , -0.07306654,
       -0.16288763,  0.12160088,  0.11416717,  0.17378834, -0.04

In [3605]:
# import pickle

import ast
import pandas as pd
import numpy as np

from worde4mde import load_embeddings
import re

import os
import openai
from openai import OpenAI
import json

import networkx as nx
import sys


In [3606]:
from dotenv import load_dotenv

# %%

load_dotenv()

openai_key = os.getenv('openai_key')
os.environ['OPENAI_API_KEY'] = openai_key

## Input

In [3607]:
##### update here to change input file directory ######
##### update here to change input file directory ######
##### update here to change input file directory ######
##### update here to change input file directory ######

instructor_dir = "results/"

# 01_LabTracker
# 02_CelO
# 03_TSS
# 04_SHAS
# 05_OTS
# 06_Block
# 07_Tile
# 08_HBMS
problem_id = "01_LabTracker"

# location of reference domain model is stored. It is in .txt format
instructor_input = instructor_dir + problem_id + ".txt"

# gpt-4o-mini
# gpt-4
# gpt-4o
llm_id = "gpt-4"

# yujing
# sherpa
method_id = "yujing"

student_dir = instructor_dir + llm_id + '/' + method_id + '/' + problem_id + '/'
if not os.path.exists(student_dir):
    # Create the directory
    os.makedirs(student_dir)
    
# location of generated domain model is saved. The file should be in .txt    
student_input = student_dir + problem_id + ".txt"
student_input
student_out_dir = student_dir + "result/"
student_out_dir

'results/gpt-4/sherpa/08_HBMS/result/'

In [3608]:
# First of all, you need to load the embeddings (currently supported: 'sgram-mde' and 'glove-mde')
sgram_mde = load_embeddings('sgram-mde')
# glove = load_embeddings('glove-mde')
# sgram_mde["id"]

In [3609]:
# word = 'id'
# sgram_mde.most_similar(positive=[word])

In [3610]:
def inEmbedding(word, embedding):
  try:
    return True, embedding[word]
  except:
    return False, None

In [3611]:

def splitCamelCase(word):
  splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', word)).split()
  return splitted

splitCamelCase("device ID")

['device', 'ID']

In [3612]:
splitCamelCase("CommandStatus")

['Command', 'Status']

In [3613]:
# client = OpenAI()

# def run_llm(prompt, model ="gpt-3.5-turbo"):
#   response = client.chat.completions.create(
#     model=model,
#     messages=[
#       {"role": "user", "content": prompt},

#     ]
#   )
#   return response.choices[0].message.content

In [3614]:
client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [3615]:
def get_all_info(class_index, class_nodes, list_of_classes, list_edges, need_edge = True):
  result = ''
  result +=  list_of_classes[class_index] + "\n"
  node = class_nodes[class_index]
  if "abstract" in node:
      node = node.replace("abstract", "").strip()
  if "Abstract" in node:
      node = node.replace("Abstract", "").strip()
  if need_edge:
    for edge in list_edges:
      element = [i.strip() for i in edge.split()]
      if node in element:
        result += edge + "\n"

  return  result

In [3616]:
def match_classes(raw_1, raw_2, dict_attr, thresh = 0.5, verbose = False):
  # raw_1: nodes from reference solution
  # raw_2: nodes from student solution
  # dict_attr: dictionary for embeddings
  # thresh for subst cost
  # map class lists raw_1 and raw_2
  print(dict_attr)
  def node_subst_cost_attr(node1, node2):
    # threshod as 0.45
    if dict_attr[node1["name"]][node2["name"]] < thresh:
      return 3
    else:
      return 1-dict_attr[node1["name"]][node2["name"]]
    


  G_att_1 = nx.Graph()
  G_att_2 = nx.Graph()
  for node in raw_1:
    G_att_1.add_node(node, name = node)

  for node in raw_2:
    G_att_2.add_node(node, name = node)

  for v in nx.optimize_edit_paths(G_att_1, G_att_2, node_subst_cost=node_subst_cost_attr, edge_match=None, timeout = 20):
      minv = v
  # minv

  return minv

# Group

In [3617]:
class Grader():
  def __init__(self):
    self.ref = None
    self.stu = None

In [3618]:
grader = Grader()

## Class

### Define metadata

In [3619]:
class ModelInfo():

  def __init(self):
    self.cls_atr = {}
    self.cls_name = []
    self.raw_dsl = []
    self.rel = None

In [3620]:
ref_cls = ModelInfo()
stu_cls = ModelInfo()

In [3621]:
grader.ref = ref_cls
grader.stu = stu_cls

In [3622]:
with open(instructor_input, 'r') as file:
    # Read the entire content of the file
    instructor_all = file.read()

with open(student_input, 'r') as file:
    # Read the entire content of the file
    student_all = file.read()

In [3623]:
print(instructor_all)

Enumerations:
RoomType(SINGLE, DOUBLE, TWIN)
BookingStatus(FINALIZED, CONFIRMED, CANCELLED_BY_HBMS, PRELIMINARY, CANCELLED_BY_TRAVELLER, CANCELLED_BY_HOTEL)
PaymentType (PREPAID, PAID_AT_HOTEL)

Classes:
HotelBookingManager()
HotelChain()

Traveller (string name, int reliabilityRating)

BillingInformation(string companyName, string address)
TravelPreference(string preference, string value)
Hotel (string city, string country, string area)
abstract Offer()
SpecialOffer()
RegularOffer()
TravelSearch()
SearchInfo()
RoomAvailability()
abstract TripInfo(int numberOfRooms, RoomType roomType, date arrivalDate, date departureDate, float budgetPerNight)
BookingInfo()
Booking (int bookingId, date cancellationDeadline, string creditCardNumber, BookingStatus bookingStatus, PaymentType paymentType, date confirmationDate, float paidAmount)


Relationships:



In [3624]:
print(student_all)

Enumerations:


Classes:

HBMS(string city, date arrivalDate, date departureDate, int numberOfRooms, string roomType, int hotelRating, float budget, boolean travelPreferences)
BusinessTraveller(string name, string companyName, string address, boolean travelPreferences, int reliabilityRating)



Hotel(string name, string address, int hotelRating, string roomType, boolean availability)

SpecialOffer(float price, string cityArea, int hotelRating, string keyPreferences)



Room(string type, int number)


Booking(string city, date arrivalDate, date departureDate, int numberOfRooms, string roomType, float price, string uniqueBookingIdentifier, float financialCompensation)
AccommodationDeal(string city, date arrivalDate, date departureDate, int numberOfRooms, string roomType, int hotelRating, float budget)
Confirmation(boolean confirmed, date confirmationDate)


Relationships:



In [3625]:
shas_sol = instructor_all.split("Relationships:")[0]
stu_sol_g =  student_all.split("Relationships:")[0] # student solution

In [3626]:
ref_classes_raw = []
tmp = shas_sol.strip().splitlines()
ref_classes_raw = [i.strip() for i in tmp if len(i) > 0]
ref_classes_raw

reference_class = []
tmp = shas_sol.strip().splitlines()
reference_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]

In [3627]:
stu_classes_raw = []
tmp = stu_sol_g.strip().splitlines()
stu_classes_raw = [i.strip() for i in tmp if len(i) > 0]
stu_classes_raw

tmp = stu_sol_g.strip().splitlines()
stu_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]
stu_class

['Enumerations:',
 'Classes:',
 'HBMS',
 'BusinessTraveller',
 'Hotel',
 'SpecialOffer',
 'Room',
 'Booking',
 'AccommodationDeal',
 'Confirmation']

In [3628]:
ref_attributes = {}
enum_index = ref_classes_raw.index("Enumerations:")
regular_index = ref_classes_raw.index("Classes:")
for cla, dsl in zip(reference_class, ref_classes_raw):
  if dsl == "Enumerations:" or dsl == "Classes:":
    continue
  else:
    index = ref_classes_raw.index(dsl)
    if index > regular_index:
      class_type = "regular"
      if "abstract" in dsl:
        class_type = "abstract"
    else:
      class_type = "enum"
    ref_attributes[cla] = {"score": 0, "type":class_type, "dsl":dsl, "counterpart": None, "attributes": {}}

    attributes = dsl.split("(")[1][:-1].split(",")
    for attr in attributes:
      attr = attr.strip()
      if len(attr) > 0:
        ref_attributes[cla]["attributes"][attr] = {"score":0, "counterpart": None}

In [3629]:
reference_class.remove("Enumerations:")
reference_class.remove("Classes:")

ref_classes_raw.remove("Enumerations:")
ref_classes_raw.remove("Classes:")

In [3630]:
ref_cls.cls_name = reference_class
ref_cls.raw_dsl = ref_classes_raw
ref_cls.cls_atr = ref_attributes

In [3631]:
stu_attributes = {}
enum_index = stu_classes_raw.index("Enumerations:")
regular_index = stu_classes_raw.index("Classes:")

for cla, dsl in zip(stu_class, stu_classes_raw):
  if dsl == "Enumerations:" or dsl == "Classes:":
    continue
  else:
    index = stu_classes_raw.index(dsl)
    if index > regular_index:
      class_type = "regular"
      if "abstract" in dsl:
        class_type = "abstract"
    else:
      class_type = "enum"

    stu_attributes[cla] = {"score": 0, "type":class_type, "dsl":dsl, "counterpart": None, "attributes": {}}

    print(dsl)
    attributes = dsl.split("(")[1][:-1].split(",")
    for attr in attributes:
      attr = attr.strip()
      if len(attr) > 0:
        stu_attributes[cla]["attributes"][attr] = {"score":0, "counterpart": None}

HBMS(string city, date arrivalDate, date departureDate, int numberOfRooms, string roomType, int hotelRating, float budget, boolean travelPreferences)
BusinessTraveller(string name, string companyName, string address, boolean travelPreferences, int reliabilityRating)
Hotel(string name, string address, int hotelRating, string roomType, boolean availability)
SpecialOffer(float price, string cityArea, int hotelRating, string keyPreferences)
Room(string type, int number)
Booking(string city, date arrivalDate, date departureDate, int numberOfRooms, string roomType, float price, string uniqueBookingIdentifier, float financialCompensation)
AccommodationDeal(string city, date arrivalDate, date departureDate, int numberOfRooms, string roomType, int hotelRating, float budget)
Confirmation(boolean confirmed, date confirmationDate)


In [3632]:
stu_class.remove("Enumerations:")
stu_class.remove("Classes:")

stu_classes_raw.remove("Enumerations:")
stu_classes_raw.remove("Classes:")

In [3633]:
stu_cls.cls_name = stu_class
stu_cls.raw_dsl = stu_classes_raw
stu_cls.cls_atr = stu_attributes

## Edges

### reference and student solutions

In [3634]:
shas_edges = instructor_all.split("Relationships:")[1]
shas_G2_edges =  student_all.split("Relationships:")[1] # student solution

In [3635]:
shas_edges

'\n'

In [3636]:
shas_edges

'\n'

In [3637]:
shas_edges = shas_edges.replace("0..*","*")

In [3638]:
shas_G2_edges = shas_G2_edges.replace("0..*","*")

In [3639]:
tmp = shas_edges.strip().splitlines()
ref_edges = [i.strip() for i in tmp if len(i) > 0]

for i in ref_edges:
  length = len(i.split())
  if length == 5 or length == 3:
    pass
  else:
    print("Reference Length error:", i)

tmp = shas_G2_edges.strip().splitlines()
stu_edges = [i.strip() for i in tmp if len(i) > 0]

for i in stu_edges:
  length = len(i.split())
  if length == 5 or length == 3:
    pass
  else:
    print("Student Length error:", i)



In [3640]:
class Relationship:
  def __init__(self):
    self.rels = [] # list of dict{}
    self.raw_dsl = []
  # find first non-matched or matched reltionshipi given dsl
  def find_relation(self, dsl, matched = False):
    for index, i in enumerate(self.rels):
      if i["dsl"] == dsl and i["counterpart"] is None:
        return index, i

    return None

In [3641]:
ref_edges_obj = Relationship()
stu_edges_obj = Relationship()


ref_edges_dict = []
for e in ref_edges:
  ref_edges_dict.append({"dsl":e,"score":0, "counterpart": None})

ref_edges_obj.rels = ref_edges_dict
ref_edges_obj.raw_dsl = ref_edges

stu_edges_dict = []
for e in stu_edges:
  stu_edges_dict.append({"dsl":e,"score":0, "counterpart": None})

stu_edges_obj.rels = stu_edges_dict
stu_edges_obj.raw_dsl = stu_edges

In [3642]:
# link to classInfo object
ref_cls.rel = ref_edges_obj
stu_cls.rel = stu_edges_obj

In [3643]:
edges = [ref_edges_obj, stu_edges_obj]

## Getting embedding

In [3644]:
def removeChars(text, chars):
    for c in chars:
        text = text.replace(c,"")
    return text


In [3645]:
def get_mde_embedding(text, embedding):
  # >>> get_mde_embedding("WhatDevice", sgram_mde)
  words = splitCamelCase(removeChars(text, ["(",")",","]))
  lowercase_list = [s.lower() for s in words]
  counter = 0
  emb = np.zeros(300)
  for w in lowercase_list:
    try:
      emb += embedding[w]
      counter += 1
    except:
      # if failed in emb the complete word, embed the token
      tmp = np.zeros(300)
      for char in list(w):

        tmp += embedding[char]
      emb += (tmp / len(w))
      counter += 1
  return emb / counter

In [3646]:
text = "AutomationStatus (Created, Edited, Activated, Deactivated)"
words = splitCamelCase(removeChars(text, ["(",")",","]))
words

['Automation', 'Status', 'Created', 'Edited', 'Activated', 'Deactivated']

In [3647]:
def cosine_distance(emb_i, emb_j):
  return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

## Stage 1.1 Class name mapping

In [3648]:
# get name embedding
# all info
similarity_mde = []
mde_embedding = sgram_mde
threshold = 0.7
similarity_mde_dsl = []
percentage = 0.8
for index, node in enumerate(ref_cls.cls_name):
  cls = node.split()[-1].strip() # get the class name, remove abstract key word
  
  mde_emb_i = get_mde_embedding(cls, mde_embedding)
  mde_emb_i_dsl = get_mde_embedding(ref_cls.cls_atr[node]["dsl"], mde_embedding)

  pair = []
  mde_pair = []
  for j, stu_node in enumerate(stu_cls.cls_name):
    cls = stu_node.split()[-1].strip() # get the class name, remove abstract key word
    print()
    mde_emb_j = get_mde_embedding(cls, mde_embedding)
    mde_emb_j_dsl = get_mde_embedding(stu_cls.cls_atr[stu_node]["dsl"], mde_embedding)

    mde_sim = cosine_distance(mde_emb_i, mde_emb_j)
    mde_sim_dsl = cosine_distance(mde_emb_i_dsl, mde_emb_j_dsl)

    mde_pair.append(mde_sim * percentage + (1-percentage) * mde_sim_dsl)

  # apply third quartile
  similarity_mde.append(mde_pair)

dict_sim_word = {}
for i in range(len(ref_cls.cls_name)):
  dict_sim_word[ref_cls.cls_name[i]] = {}
  for j in range(len(stu_cls.cls_name)):
    dict_sim_word[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_mde[i][j]

In [3649]:
for key in dict_sim_word:
    exact_match = False
    exact_cls = ""
    for cls_2 in dict_sim_word[key]:
        if dict_sim_word[key][cls_2] > 0.99:
            exact_match = True
            exact_cls = cls_2
            print(cls_2)
            break
    
    if exact_match:    # keep the exact match, others to 0
        print("update")
        for cls_2 in dict_sim_word[key]:
            if cls_2 !=  exact_cls:
                dict_sim_word[key][cls_2] = 0

similarity_mde = []
for i in dict_sim_word:
    tmp = []
    for j in dict_sim_word[i]:
        tmp.append(dict_sim_word[i][j])

    similarity_mde.append(tmp)

In [3650]:
import pandas as pd

df = pd.DataFrame(index=reference_class, columns=stu_class, data=similarity_mde)
df

,HBMS,BusinessTraveller,Hotel,SpecialOffer,Room,Booking,AccommodationDeal,Confirmation
RoomType,0.252150,0.270693,0.400047,0.290126,0.803402,0.397912,0.313996,0.233427
BookingStatus,0.306146,0.434784,0.455739,0.311174,0.365694,0.792375,0.361364,0.387458
PaymentType,0.189935,0.392296,0.340370,0.403015,0.247521,0.483809,0.387861,0.421696
HotelBookingManager,0.257302,0.454526,0.762139,0.317623,0.382589,0.758401,0.418199,0.283701
HotelChain,0.214408,0.385508,0.784038,0.294366,0.283872,0.416303,0.382899,0.212599
Traveller,0.236027,0.856694,0.441611,0.341422,0.290531,0.428739,0.354634,0.323995
BillingInformation,0.196264,0.569551,0.414963,0.393410,0.320733,0.469100,0.392562,0.373408
TravelPreference,0.207524,0.544683,0.442040,0.367817,0.299061,0.488069,0.445356,0.326849
Hotel,0.254894,0.452910,0.965310,0.326743,0.389864,0.561671,0.440926,0.213744
abstract Offer,0.151255,0.284774,0.247690,0.669884,0.167956,0.206760,0.288595,0.249763


### Stage 1.1.1 Mapping (enum / regular + abstract)

In [3651]:
ref_enum = []
ref_reg_cls = []
stu_enum = []
stu_reg_cls = []

for enum in ref_cls.cls_name:
  if ref_cls.cls_atr[enum]["type"] == "enum":
    ref_enum.append(enum)
  else:
    ref_reg_cls.append(enum)

for enum in stu_cls.cls_name:
  if stu_cls.cls_atr[enum]["type"] == "enum":
    stu_enum.append(enum)
  else:
    stu_reg_cls.append(enum)

In [3652]:
enum_mapping = match_classes(ref_enum, stu_enum, dict_sim_word, thresh = 0.5)
enum_mapping[0]

{'RoomType': {'HBMS': 0.25214963468283036, 'BusinessTraveller': 0.2706933395490135, 'Hotel': 0.4000472521679917, 'SpecialOffer': 0.29012563382361956, 'Room': 0.8034022909699896, 'Booking': 0.3979115522407458, 'AccommodationDeal': 0.31399624447308255, 'Confirmation': 0.23342720387181948}, 'BookingStatus': {'HBMS': 0.30614621398713976, 'BusinessTraveller': 0.43478413371069535, 'Hotel': 0.4557390090286323, 'SpecialOffer': 0.31117415882730476, 'Room': 0.3656940427093817, 'Booking': 0.7923749400747551, 'AccommodationDeal': 0.3613638227235778, 'Confirmation': 0.38745772545580764}, 'PaymentType': {'HBMS': 0.18993509086788077, 'BusinessTraveller': 0.3922960749790374, 'Hotel': 0.3403704145677425, 'SpecialOffer': 0.4030145571705338, 'Room': 0.24752058979651503, 'Booking': 0.48380851409387005, 'AccommodationDeal': 0.387861232112681, 'Confirmation': 0.4216955004517514}, 'HotelBookingManager': {'HBMS': 0.25730239285645806, 'BusinessTraveller': 0.45452578988646664, 'Hotel': 0.76213901015159, 'Specia

[('RoomType', None), ('BookingStatus', None), ('PaymentType', None)]

In [3653]:
cls_mapping = match_classes(ref_reg_cls, stu_reg_cls, dict_sim_word, thresh = 0.5)
cls_mapping[0]

{'RoomType': {'HBMS': 0.25214963468283036, 'BusinessTraveller': 0.2706933395490135, 'Hotel': 0.4000472521679917, 'SpecialOffer': 0.29012563382361956, 'Room': 0.8034022909699896, 'Booking': 0.3979115522407458, 'AccommodationDeal': 0.31399624447308255, 'Confirmation': 0.23342720387181948}, 'BookingStatus': {'HBMS': 0.30614621398713976, 'BusinessTraveller': 0.43478413371069535, 'Hotel': 0.4557390090286323, 'SpecialOffer': 0.31117415882730476, 'Room': 0.3656940427093817, 'Booking': 0.7923749400747551, 'AccommodationDeal': 0.3613638227235778, 'Confirmation': 0.38745772545580764}, 'PaymentType': {'HBMS': 0.18993509086788077, 'BusinessTraveller': 0.3922960749790374, 'Hotel': 0.3403704145677425, 'SpecialOffer': 0.4030145571705338, 'Room': 0.24752058979651503, 'Booking': 0.48380851409387005, 'AccommodationDeal': 0.387861232112681, 'Confirmation': 0.4216955004517514}, 'HotelBookingManager': {'HBMS': 0.25730239285645806, 'BusinessTraveller': 0.45452578988646664, 'Hotel': 0.76213901015159, 'Specia

[('HotelBookingManager', None),
 ('HotelChain', None),
 ('Traveller', 'BusinessTraveller'),
 ('BillingInformation', None),
 ('TravelPreference', None),
 ('Hotel', 'Hotel'),
 ('abstract Offer', None),
 ('SpecialOffer', 'SpecialOffer'),
 ('RegularOffer', None),
 ('TravelSearch', None),
 ('SearchInfo', None),
 ('RoomAvailability', 'Room'),
 ('abstract TripInfo', None),
 ('BookingInfo', None),
 ('Booking', 'Booking'),
 (None, 'HBMS'),
 (None, 'AccommodationDeal'),
 (None, 'Confirmation')]

In [3654]:
mapping = enum_mapping[0] + cls_mapping[0]

for map in mapping:
  if map[0] and map[1]:
    ref_cls.cls_atr[map[0]]['score'] = 1
    ref_cls.cls_atr[map[0]]['counterpart'] = map[1]

    stu_cls.cls_atr[map[1]]['score'] = 1
    stu_cls.cls_atr[map[1]]['counterpart'] = map[0]

    if ref_cls.cls_atr[map[0]]["type"] !=  stu_cls.cls_atr[map[1]]["type"]:
      # type mismatch
      ref_cls.cls_atr[map[0]]['score'] = 0.5
      stu_cls.cls_atr[map[1]]["score"] = 0.5

In [3655]:
ref_cls.cls_atr

{'RoomType': {'score': 0,
  'type': 'enum',
  'dsl': 'RoomType(SINGLE, DOUBLE, TWIN)',
  'counterpart': None,
  'attributes': {'SINGLE': {'score': 0, 'counterpart': None},
   'DOUBLE': {'score': 0, 'counterpart': None},
   'TWIN': {'score': 0, 'counterpart': None}}},
 'BookingStatus': {'score': 0,
  'type': 'enum',
  'dsl': 'BookingStatus(FINALIZED, CONFIRMED, CANCELLED_BY_HBMS, PRELIMINARY, CANCELLED_BY_TRAVELLER, CANCELLED_BY_HOTEL)',
  'counterpart': None,
  'attributes': {'FINALIZED': {'score': 0, 'counterpart': None},
   'CONFIRMED': {'score': 0, 'counterpart': None},
   'CANCELLED_BY_HBMS': {'score': 0, 'counterpart': None},
   'PRELIMINARY': {'score': 0, 'counterpart': None},
   'CANCELLED_BY_TRAVELLER': {'score': 0, 'counterpart': None},
   'CANCELLED_BY_HOTEL': {'score': 0, 'counterpart': None}}},
 'PaymentType': {'score': 0,
  'type': 'enum',
  'dsl': 'PaymentType (PREPAID, PAID_AT_HOTEL)',
  'counterpart': None,
  'attributes': {'PREPAID': {'score': 0, 'counterpart': None},


## Stage 1.2 Class Mapping (with all info)

In [3656]:
ref_classes = []
stu_classes = []

for cls in ref_cls.cls_name:
  if ref_cls.cls_atr[cls]["counterpart"] == None:
    ref_classes.append(cls)

for cls in stu_cls.cls_name:
  if stu_cls.cls_atr[cls]["counterpart"] == None:
    stu_classes.append(cls)

In [3657]:
# get name embedding
# all info
similarity_all = []
mde_embedding = sgram_mde
threshold = 0.7




# reuse saved embedding result...
# may cause problem
# if os.path.exists(student_dir + "ada_embedding.txt"):
#   pass
  
  # with open(student_dir + "ada_embedding.txt", 'r') as json_file:
  #   dict_sim_all = json.load(json_file)

if False:
  pass

else:
  for index, node in enumerate(ref_cls.cls_name):
    text_1 = get_all_info(index, ref_cls.cls_name, ref_cls.raw_dsl, ref_cls.rel.raw_dsl, True)
    emb_i = get_embedding(text_1)
    mde_pair = []
    for j, stu_node in enumerate(stu_cls.cls_name):
      text_2 = get_all_info(j, stu_cls.cls_name, stu_cls.raw_dsl, stu_cls.rel.raw_dsl, True)
      emb_j = get_embedding(text_2)
      mde_sim = cosine_distance(emb_i, emb_j)
      mde_pair.append(mde_sim)

    # apply third quartile
    similarity_all.append(mde_pair)

  dict_sim_all = {}
  for i in range(len(ref_cls.cls_name)):
    dict_sim_all[ref_cls.cls_name[i]] = {}
    for j in range(len(stu_cls.cls_name)):
      dict_sim_all[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_all[i][j]





In [3658]:
with open(student_dir + "ada_embedding.txt", 'w') as json_file:
    json.dump(dict_sim_all, json_file)

In [3659]:
student_dir

'results/gpt-4/sherpa/08_HBMS/'

In [3660]:
ref_classes

['RoomType',
 'BookingStatus',
 'PaymentType',
 'HotelBookingManager',
 'HotelChain',
 'BillingInformation',
 'TravelPreference',
 'abstract Offer',
 'RegularOffer',
 'TravelSearch',
 'SearchInfo',
 'abstract TripInfo',
 'BookingInfo']

In [3661]:
stu_classes

['HBMS', 'AccommodationDeal', 'Confirmation']

In [3662]:
dict_sim_all

{'RoomType': {'HBMS': 0.804840725104826,
  'BusinessTraveller': 0.7811249629928283,
  'Hotel': 0.8408345123820181,
  'SpecialOffer': 0.8013737019585344,
  'Room': 0.877537021230031,
  'Booking': 0.8101290769682334,
  'AccommodationDeal': 0.8119374220133271,
  'Confirmation': 0.7640029540639278},
 'BookingStatus': {'HBMS': 0.8183165492638729,
  'BusinessTraveller': 0.7792535081412233,
  'Hotel': 0.7993107361723563,
  'SpecialOffer': 0.7732534754614532,
  'Room': 0.777040305224274,
  'Booking': 0.8125943053414727,
  'AccommodationDeal': 0.775603114484755,
  'Confirmation': 0.797169465991154},
 'PaymentType': {'HBMS': 0.7817584628370285,
  'BusinessTraveller': 0.755398489554848,
  'Hotel': 0.7977326556764767,
  'SpecialOffer': 0.8007195931442392,
  'Room': 0.7911308964613673,
  'Booking': 0.8001536047825836,
  'AccommodationDeal': 0.7911799472930916,
  'Confirmation': 0.7600161695587908},
 'HotelBookingManager': {'HBMS': 0.8377303097667167,
  'BusinessTraveller': 0.8451244448337505,
  'Ho

In [3663]:
mapping = match_classes(ref_classes, stu_classes, dict_sim_all, 0.8)
mapping[0]

{'RoomType': {'HBMS': 0.804840725104826, 'BusinessTraveller': 0.7811249629928283, 'Hotel': 0.8408345123820181, 'SpecialOffer': 0.8013737019585344, 'Room': 0.877537021230031, 'Booking': 0.8101290769682334, 'AccommodationDeal': 0.8119374220133271, 'Confirmation': 0.7640029540639278}, 'BookingStatus': {'HBMS': 0.8183165492638729, 'BusinessTraveller': 0.7792535081412233, 'Hotel': 0.7993107361723563, 'SpecialOffer': 0.7732534754614532, 'Room': 0.777040305224274, 'Booking': 0.8125943053414727, 'AccommodationDeal': 0.775603114484755, 'Confirmation': 0.797169465991154}, 'PaymentType': {'HBMS': 0.7817584628370285, 'BusinessTraveller': 0.755398489554848, 'Hotel': 0.7977326556764767, 'SpecialOffer': 0.8007195931442392, 'Room': 0.7911308964613673, 'Booking': 0.8001536047825836, 'AccommodationDeal': 0.7911799472930916, 'Confirmation': 0.7600161695587908}, 'HotelBookingManager': {'HBMS': 0.8377303097667167, 'BusinessTraveller': 0.8451244448337505, 'Hotel': 0.8430153179829974, 'SpecialOffer': 0.82030

[('RoomType', None),
 ('BookingStatus', None),
 ('PaymentType', None),
 ('HotelBookingManager', 'HBMS'),
 ('HotelChain', None),
 ('BillingInformation', None),
 ('TravelPreference', None),
 ('abstract Offer', None),
 ('RegularOffer', None),
 ('TravelSearch', None),
 ('SearchInfo', None),
 ('abstract TripInfo', 'AccommodationDeal'),
 ('BookingInfo', None),
 (None, 'Confirmation')]

In [3664]:
for map in mapping[0]:
  if map[0] and map[1]:
    ref_cls.cls_atr[map[0]]['score'] = 1
    ref_cls.cls_atr[map[0]]['counterpart'] = map[1]

    stu_cls.cls_atr[map[1]]['score'] = 1
    stu_cls.cls_atr[map[1]]['counterpart'] = map[0]

    if ref_cls.cls_atr[map[0]]["type"] !=  stu_cls.cls_atr[map[1]]["type"]:
      # type mismatch
      ref_cls.cls_atr[map[0]]['score'] = 0.5
      stu_cls.cls_atr[map[1]]["score"] = 0.5

## Stage 2.1 Attribute mapping

In [3665]:
def get_cosine_disntance(text1, text2, embedding):
    avg = np.sum(embedding[text1], axis=0)
    emb_i = avg / len(text1)

    avg = np.sum(embedding[text2], axis=0)
    emb_j = avg / len(text2)

    return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

In [3666]:
def create_cosine_similarity_dict(attr_1, attr_2, embedding):
  # attr_1: list[list] split camel case into separated case
  # attr_2: list[list] split camel case into separated case
  # embedding:
  # raw_1: list[str] camcel case attributes
  # raw_2: list[str] camcel case attributes

  # create cosine distance between two attributes with embeeding

  # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
  similarities = []
  for att in attr_1:
    emb_i = get_mde_embedding(att, embedding)
    pair = []
    for attribute in attr_2:
      emb_j = get_mde_embedding(attribute, embedding)

      sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
      pair.append(sim)

    similarities.append(pair)

  dict_attr = {}
  for i in range(len(attr_1)):
    dict_attr[attr_1[i]] = {}
    for j in range(len(attr_2)):
      dict_attr[attr_1[i]][attr_2[j]] = similarities[i][j]

  return dict_attr

In [3667]:
def match_attributes(raw_1, raw_2, dict_attr, threshold = 0.5):
  # map attributes lists raw_1 and raw_2
  # >>> mapAttributes(['deviceStatus'], ['id'], dic)[0]
  def node_subst_cost_attr(node1, node2):
    # threshod as 0.45
    if dict_attr[node1["name"]][node2["name"]] < threshold:
      return 3
    else:
      return 1-dict_attr[node1["name"]][node2["name"]]

  G_att_1 = nx.Graph()
  G_att_2 = nx.Graph()
  for node in raw_1:
    G_att_1.add_node(node, name = node)

  for node in raw_2:
    G_att_2.add_node(node, name = node)

  for v in nx.optimize_edit_paths(G_att_1, G_att_2, node_subst_cost=node_subst_cost_attr, edge_match=None, timeout = 20):
      minv = v
  # minv

  return minv

In [3668]:
def check_attributes_type(attr_1, attr_2, ref_dict, stu_dict):
  # given two attribute, compare whether their type can match
  attr_1 = attr_1.split() # from reference solution do not need to check it
  attr_2 = attr_2.split()

  # if enumerate literal and attributes
  # if len(attr_1) != len(attr_2):
  #   return 0.5, 0.5

  # if an attributes use a regular class as its type
  if len(attr_2) == 2:
    t = attr_2[0].strip()
    cls = stu_dict.get(t, None)
    if cls is not None and (stu_dict[t]['type'] == "regular" or stu_dict[t]['type'] == "abstract"):
      return 0.5, 0.5
    
  if len(attr_1) == 2 or len(attr_2) == 2:
    t = attr_1[0].strip()
    cls = ref_dict.get(t, None)
    # print(cls)
    t_2 = attr_2[0].strip()
    cls_2 = stu_dict.get(t_2, None)   
    # print(cls_2) 
    if cls is not None:
      if ref_dict[t]['counterpart'] != t_2:
        return 0.5, 0.5
    if cls_2 is not None:
      if stu_dict[t_2]['counterpart'] != t:
        return 0.5, 0.5
    
  return 1,1

In [3669]:
x =check_attributes_type(('CommandStatus commandStatus', 'ControllerStatus status')[0], ('CommandStatus commandStatus', 'ControllerStatus status')[1], ref_cls.cls_atr, stu_cls.cls_atr)
x

(1, 1)

### Stage 2.1.1 Attribute <-> Attribute (mapped classes)

In [3670]:
# map classes
pairs = []

# get matched pairs
for key in ref_cls.cls_atr:
  if ref_cls.cls_atr[key]["counterpart"] != None:
    pair = [key, ref_cls.cls_atr[key]["counterpart"]]
    pairs.append(pair)

for pair in pairs:
  if pair[0] and pair[1]:
    print(pair)

    # map attributes:
    # matched_ref = set()
    # matched_attr = set()

    raw_1 = []
    for attributes in ref_cls.cls_atr[pair[0]]["attributes"]:
      raw_1.append(attributes)

    raw_2 = []
    for attributes in stu_cls.cls_atr[pair[1]]["attributes"]:
      raw_2.append(attributes)

    cos_dict = create_cosine_similarity_dict(raw_1, raw_2, sgram_mde)
    mappings = match_attributes(raw_1, raw_2, cos_dict)[0]
    print("mapping", mappings)

    for mapping in mappings:
      if mapping[0] != None and mapping[1] != None:
        scores = check_attributes_type(mapping[0], mapping[1], ref_cls.cls_atr, stu_cls.cls_atr)
        ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]['score'] = scores[0]
        ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]["counterpart"] = (mapping[1], pair[1])
        stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]['score'] = scores[1]
        stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]["counterpart"] =  (mapping[0], pair[0])

    print("="*20)

['HotelBookingManager', 'HBMS']
mapping [(None, 'string city'), (None, 'date arrivalDate'), (None, 'date departureDate'), (None, 'int numberOfRooms'), (None, 'string roomType'), (None, 'int hotelRating'), (None, 'float budget'), (None, 'boolean travelPreferences')]
['Traveller', 'BusinessTraveller']
mapping [('string name', 'string name'), ('int reliabilityRating', 'int reliabilityRating'), (None, 'string companyName'), (None, 'string address'), (None, 'boolean travelPreferences')]
['Hotel', 'Hotel']
mapping [('string city', 'string address'), ('string country', 'string name'), ('string area', 'string roomType'), (None, 'int hotelRating'), (None, 'boolean availability')]
['SpecialOffer', 'SpecialOffer']
mapping [(None, 'float price'), (None, 'string cityArea'), (None, 'int hotelRating'), (None, 'string keyPreferences')]
['RoomAvailability', 'Room']
mapping [(None, 'string type'), (None, 'int number')]
['abstract TripInfo', 'AccommodationDeal']
mapping [('int numberOfRooms', 'int number

### Stage 2.1.2 Attribute <-> Attribute (between any classes)

In [3671]:
def create_cosine_similarity_list(attr_1, attr_2, embedding):
  # attr_1: list[list] split camel case into separated case
  # attr_2: list[list] split camel case into separated case
  # embedding:
  # raw_1: list[str] camcel case attributes
  # raw_2: list[str] camcel case attributes

  # create cosine distance between two attributes with embeeding

  # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
  similarities = []
  for att in attr_1:
    emb_i = get_mde_embedding(att, embedding)
    pair = []
    for attribute in attr_2:
      emb_j = get_mde_embedding(attribute, embedding)

      sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
      pair.append(sim)

    similarities.append(pair)

  return similarities

In [3672]:
def combine_two_dict(list_1, list_2, atr_1, atr_2, cls_1, cls_2, weight_1 = 0.9, weight_2=0.1):
  # list_1: similarity between cls
  # list_2: cos similarity between attributes
  if weight_1 + weight_2 != 1:
    raise ValueError("weight_1 + weight_2 != 1")
  result = {}
  for i in range(len(atr_1)):
    atr_r = atr_1[i]
    cls_r = cls_1[i]
    tmp_ref = (atr_r, cls_r)
    result[tmp_ref] = {}
    for j in range(len(atr_2)):
      atr_s = atr_2[j]
      cls_s = cls_2[j]
      tmp_stu = (atr_s, cls_s)

      result[tmp_ref][tmp_stu] = weight_1 * list_1[i][j] + weight_2 * list_2[i][j]

  return result


In [3673]:
# attr between any classes

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
  for attributes in ref_cls.cls_atr[cls]["attributes"]:
    if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_1.append(attributes)
      ref_source.append(cls)
      tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get attributes on student sides
for cls in stu_cls.cls_atr:
  for attributes in stu_cls.cls_atr[cls]["attributes"]:
    if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_2.append(attributes)
      stu_source.append(cls)
      tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)


    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = min(scores[0], 0.5)
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = mapping[1]

    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = min(scores[1], 0.5)
    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] =  mapping[0]

    if ref_cls.cls_atr[mapping[0][1]]['counterpart'] == mapping[1][1]:
      ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = min(scores[0], 1)

      stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = min(scores[1], 1)
print("="*20)

(('SINGLE', 'RoomType'), None)
(('DOUBLE', 'RoomType'), None)
(('TWIN', 'RoomType'), None)
(('FINALIZED', 'BookingStatus'), None)
(('CONFIRMED', 'BookingStatus'), ('boolean confirmed', 'Confirmation'))
(('CANCELLED_BY_HBMS', 'BookingStatus'), None)
(('PRELIMINARY', 'BookingStatus'), None)
(('CANCELLED_BY_TRAVELLER', 'BookingStatus'), None)
(('CANCELLED_BY_HOTEL', 'BookingStatus'), ('int hotelRating', 'Hotel'))
(('PREPAID', 'PaymentType'), None)
(('PAID_AT_HOTEL', 'PaymentType'), ('int hotelRating', 'SpecialOffer'))
(('string companyName', 'BillingInformation'), ('string companyName', 'BusinessTraveller'))
(('string address', 'BillingInformation'), ('string address', 'BusinessTraveller'))
(('string preference', 'TravelPreference'), ('string keyPreferences', 'SpecialOffer'))
(('string value', 'TravelPreference'), ('string type', 'Room'))
(('string creditCardNumber', 'Booking'), ('string city', 'Booking'))
(('PaymentType paymentType', 'Booking'), None)
(None, ('string city', 'HBMS'))
(Non

## Stage 2.2.1 Attribute mapping atr -> cls

In [3674]:
#  get non-mapped attributes on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
  for attributes in ref_cls.cls_atr[cls]["attributes"]:
    if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_1.append(attributes)
      ref_source.append(cls)
      tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
    if stu_cls.cls_atr[cls]["counterpart"] == None:
      raw_2.append(cls)
      stu_source.append(cls)
      tup_s.append((cls, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = 0.5
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = (None, mapping[1][1])

    stu_cls.cls_atr[mapping[1][1]]['score'] = 0.5
    stu_cls.cls_atr[mapping[1][1]]["counterpart"] =  mapping[0]

print("="*20)


(('SINGLE', 'RoomType'), None)
(('DOUBLE', 'RoomType'), None)
(('TWIN', 'RoomType'), None)
(('FINALIZED', 'BookingStatus'), None)
(('CANCELLED_BY_HBMS', 'BookingStatus'), None)
(('PRELIMINARY', 'BookingStatus'), None)
(('CANCELLED_BY_TRAVELLER', 'BookingStatus'), None)
(('PREPAID', 'PaymentType'), None)
(('PaymentType paymentType', 'Booking'), None)
(None, ('Confirmation', 'Confirmation'))


## Stage 2.2.2 Attribute mapping cls -> atr

In [3675]:
#  get non-mapped cls on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get class on instrucotr sides
for cls in ref_cls.cls_atr:
    if ref_cls.cls_atr[cls]["counterpart"] == None:
      raw_1.append(cls)
      ref_source.append(cls)
      tup_r.append((cls, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
  for attributes in stu_cls.cls_atr[cls]["attributes"]:
    if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_2.append(attributes)
      stu_source.append(cls)
      tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
    ref_cls.cls_atr[mapping[0][1]]['score'] = 0.5
    ref_cls.cls_atr[mapping[0][1]]["counterpart"] = mapping[1]

    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = 0.5
    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] = (None, mapping[0][1])

print("="*20)


(('RoomType', 'RoomType'), ('string roomType', 'Booking'))
(('BookingStatus', 'BookingStatus'), None)
(('PaymentType', 'PaymentType'), None)
(('HotelChain', 'HotelChain'), ('int hotelRating', 'AccommodationDeal'))
(('BillingInformation', 'BillingInformation'), None)
(('TravelPreference', 'TravelPreference'), ('boolean travelPreferences', 'BusinessTraveller'))
(('abstract Offer', 'abstract Offer'), None)
(('RegularOffer', 'RegularOffer'), None)
(('TravelSearch', 'TravelSearch'), ('boolean travelPreferences', 'HBMS'))
(('SearchInfo', 'SearchInfo'), None)
(('BookingInfo', 'BookingInfo'), None)
(None, ('string city', 'HBMS'))
(None, ('date arrivalDate', 'HBMS'))
(None, ('date departureDate', 'HBMS'))
(None, ('int numberOfRooms', 'HBMS'))
(None, ('string roomType', 'HBMS'))
(None, ('int hotelRating', 'HBMS'))
(None, ('float budget', 'HBMS'))
(None, ('boolean availability', 'Hotel'))
(None, ('float price', 'SpecialOffer'))
(None, ('string cityArea', 'SpecialOffer'))
(None, ('int number', 'Ro

## Stage 3: Relationship mapping

In [3676]:
def check_relations_classes(ref_index, stu_index, ref_elements, stu_elements):
    # ref_index: index of the element in the dsl
    # stu_index: index of the element in the dsl
    # ref_elements: list[str] elements in the dsl
    # stu_elements: list[str] elements in the dsl

    ref_class_1 = ref_elements[ref_index]
    # check whether this is an abstract class
    dict_keys = list(ref_attributes.keys())
    if not ref_class_1 in dict_keys:
      abstract = "abstract " + ref_class_1
      ref_class_1 = abstract

    true_pair = ref_attributes[ref_class_1]["counterpart"]
    if isinstance(true_pair, tuple):
      boolean_1 = False
    elif true_pair == None:
      boolean_1 = False
    else:
      tmp = true_pair.replace("abstract ", "")
      boolean_1 = stu_elements[stu_index] == tmp
    return boolean_1

#### Compare rel

In [3677]:

def compare_edges(ref_e, stu_e):
  # compare if two relationships match
  # return bool, score, score

  ref_elements = ref_e.split()
  stu_elements = stu_e.split()
  # print(ref_elements, stu_elements)

  # exact match first
  # length must match
  n_ref = len(ref_elements)
  n_stu = len(stu_elements)
  if n_ref == n_stu and n_ref == 5:
    # all element must match
    boolean_1 = check_relations_classes(1,1,ref_elements,stu_elements )

    boolean_4 = check_relations_classes(4,4,ref_elements,stu_elements )

    # type
    boolean_2 = ref_elements[2] == stu_elements[2]

    # association
    boolean_0 = ref_elements[0] == stu_elements[0]

    # association
    boolean_3 = ref_elements[3] == stu_elements[3]

    if boolean_0 and boolean_1 and boolean_2 and boolean_3 and boolean_4:
      print("exact match success")
      print(ref_e, "|||", stu_e)
      return True, 1, 1

    boolean_1_4 = False
    boolean_4_1 = False
    boolean_1_4 = check_relations_classes(1,4,ref_elements,stu_elements )
    boolean_4_1 = check_relations_classes(4,1,ref_elements,stu_elements )

    # check if the relationship is fliped, this only works for associate, not containment
    if  "associate" in ref_elements[2] and "associate" in stu_elements[2]:
      # association
      boolean_0 = ref_elements[0] == stu_elements[3]

      # association
      boolean_3 = ref_elements[3] == stu_elements[0]

    if boolean_0 and boolean_1_4 and boolean_3 and boolean_4_1 and boolean_2:
      print("match success, flipped associate")
      print(ref_e, "|||", stu_e)
      return True, 1, 1

    # check if relationship is partially match
    if  (boolean_1 and boolean_4) or (boolean_1_4 and boolean_4_1):
      print("match partially success")
      print(ref_e, "|||", stu_e)
      return True, 0.5, 0.5

  if n_ref == n_stu and n_ref == 3:
    # all element must match
    boolean_0 = check_relations_classes(0,0,ref_elements,stu_elements )
    boolean_2 = check_relations_classes(2,2,ref_elements,stu_elements )

    # type
    boolean_1 = ref_elements[1] == stu_elements[1]

    if boolean_0 and boolean_1 and boolean_2:
      print("match success")
      print(ref_e, "|||", stu_e)
      # ref_dict[ref_e]["score"] = 1
      # ref_dict[ref_e]["counterpart"] = stu_e

      # stu_dict[stu_e]["score"] = 1
      # stu_dict[stu_e]["counterpart"] = ref_e
      return True, 1, 1


  return False,0, 0

In [3678]:

for i, ref_tmp in enumerate(edges[0].raw_dsl):
  matchings = []
  for j, stu_tmp in enumerate(edges[1].raw_dsl):
    if edges[0].rels[i]["counterpart"] is None and edges[1].rels[j]["counterpart"] is None:
      result = compare_edges(ref_tmp, stu_tmp)
      if result[0]:
        matching = (i, j, result[1], result[2])
        matchings.append(matching)
        # matched_stu_edges.add(j)
        # matched_ref_edges.add(i)
        # edges[0].rels[i]["score"] = result[1]
        # edges[0].rels[i]["counterpart"] = stu_tmp

        # edges[1].rels[j]["score"] = result[2]
        # edges[1].rels[j]["counterpart"] = ref_tmp
  print(matchings)
  if len(matchings) > 0:
    print()
    highest_score = 0
    matching = None

    for m in matchings:
      if m[2] > highest_score:
        matching = m
        highest_score = m[2]
    j_j = matching[1]
    i_i = matching[0]
    
    edges[0].rels[i_i]["score"] = m[2]
    edges[0].rels[i_i]["counterpart"] = edges[1].raw_dsl[j_j]

    edges[1].rels[j_j]["score"] = m[3]
    edges[1].rels[j_j]["counterpart"] = edges[0].raw_dsl[i_i]



## Result: Precision/Recall/F1

### save matching to disk

In [3679]:
if not os.path.exists(student_out_dir):
    os.makedirs(student_out_dir)

In [3680]:
# with open( student_out_dir + 'matching.pkl', 'wb') as outp:
#     pickle.dump(grader, outp, pickle.HIGHEST_PROTOCOL)

In [3681]:
student_out_dir

'results/gpt-4/sherpa/08_HBMS/result/'

In [3682]:


# as requested in comment

with open(student_out_dir + 'solution_matching.txt', 'w') as file:
     file.write(json.dumps(grader.ref.cls_atr))


with open(student_out_dir + 'student_matching.txt', 'w') as file:
     file.write(json.dumps(grader.stu.cls_atr))

with open(student_out_dir + 'solution_matching_relationship.txt', 'w') as file:
     file.write(json.dumps(grader.ref.rel.rels))


with open(student_out_dir + 'student_matching_relationship.txt', 'w') as file:
     file.write(json.dumps(grader.stu.rel.rels))

In [3683]:
student_out_dir

'results/gpt-4/sherpa/08_HBMS/result/'

###### to local

In [3684]:
# # # be careful...

# with open(student_dir + "human_eval/" + 'ref_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.ref.cls_atr))


# with open(student_dir + "human_eval/" + 'stu_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.stu.cls_atr))

# with open(student_dir + "human_eval/" + 'ref_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.ref.rel.rels))


# with open(student_dir + "human_eval/" + 'stu_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.stu.rel.rels))

In [3685]:
algo_result = {}
algo_result["class"] = {"precision": 0, "recall": 0, "f1":0}
algo_result["attribute"] = {"precision": 0, "recall": 0, "f1":0}
algo_result["relationship"] = {"precision": 0, "recall": 0, "f1":0}

### Class

In [3686]:
# recell
count = 0
recall = 0
for key in list(ref_attributes.keys()):
  count += 1
  recall += ref_attributes[key]["score"]
print(count, recall)

if count != 0:
  algo_result["class"]['recall'] = recall / count

18 8.5


In [3687]:
# precision
count = 0
precision = 0
for key in list(stu_attributes.keys()):
  count += 1
  precision += stu_attributes[key]["score"]

print(count, precision)
if count != 0:
  algo_result["class"]['precision'] = precision / count

if algo_result["class"]['recall']   + algo_result["class"]['precision'] != 0:
  algo_result["class"]['f1'] = 2* (algo_result["class"]['recall']   * algo_result["class"]['precision'] ) / ( algo_result["class"]['recall']   + algo_result["class"]['precision']  )


8 6.5


### Attribute

In [3688]:
# recall
count = 0
recall = 0
for key in list(ref_attributes.keys()):
  attrs = ref_attributes[key]["attributes"]

  for att in attrs:
    count += 1
    recall += ref_attributes[key]["attributes"][att]["score"]
print(count, recall)

if count != 0:
  algo_result["attribute"]['recall'] = recall / count

32 18.5


In [3689]:
count = 0
precision = 0
for key in list(stu_attributes.keys()):
  attrs = stu_attributes[key]["attributes"]

  for att in attrs:
    count += 1
    precision += stu_attributes[key]["attributes"][att]["score"]
print(count, precision)

if count != 0:
  algo_result["attribute"]['precision'] = precision / count

r = algo_result["attribute"]['recall']
p = algo_result["attribute"]['precision']

if r + p != 0:
  algo_result["attribute"]['f1'] = 2* (r * p) / (r + p)


41 20.5


### Relationship

In [3690]:
count = 0
recall = 0
for tmp in edges[0].rels:
  attrs = tmp["score"]

  count += 1
  recall += attrs
print(count, recall)

if count != 0:
  algo_result["relationship"]['recall'] = recall / count

0 0


In [3691]:
count = 0
precision = 0
for tmp in edges[1].rels:
  attrs = tmp["score"]

  count += 1
  precision += attrs
print(count, precision)

if count!= 0:
  algo_result["relationship"]['precision'] = precision / count
r = algo_result["relationship"]['recall']
p = algo_result["relationship"]['precision']

if r + p != 0:
  algo_result["relationship"]['f1'] = 2* (r * p ) / ( r + p )


0 0


In [3692]:

# as requested in comment

with open(student_out_dir + 'algo_result.txt', 'w') as file:
     file.write(json.dumps(algo_result))